In [2]:
from glob2 import glob
train_file_path = 'E:\\hy\\Datasets\\TUSZ\\edf\\train\\*\\*\\*\\*\\*.edf'
test_file_path = 'E:\\hy\\Datasets\\TUSZ\\edf\\dev\\*\\*\\*\\*\\*.edf'
train_edf_file_list = glob(train_file_path)
test_edf_file_list = glob(test_file_path)
print(len(train_edf_file_list))
print(len(test_edf_file_list))

4599
357


In [3]:
import numpy as np
# extract subject IDs from the file path, create python set to extract unique elements from list, convert to list again 
unique_train_epilepsy_patient_ids = list(set([x.split("\\")[-1].split("_")[0] for x in train_edf_file_list]))
unique_test_epilepsy_patient_ids = list(set([x.split("\\")[-1].split("_")[0] for x in test_edf_file_list]))
print(len(unique_train_epilepsy_patient_ids))
print(len(unique_test_epilepsy_patient_ids))

592
31


In [4]:
train_subjects_name_file_path = r'E:\hy\Datasets\TUSZ\train_4_class_corpus_subjects.txt'
test_subjects_name_file_path = r'E:\hy\Datasets\TUSZ\test_4_class_corpus_subjects.txt'
with open(train_subjects_name_file_path, 'w') as file_handler:
    for item in unique_train_epilepsy_patient_ids:
        file_handler.write("{}\n".format(item))

with open(test_subjects_name_file_path, 'w') as file_handler:
    for item in unique_test_epilepsy_patient_ids:
        file_handler.write("{}\n".format(item))

In [5]:
train_edf_file_list = glob(train_file_path)
test_edf_file_list = glob(test_file_path)
print(train_edf_file_list[0])
print(test_edf_file_list[0])
channel_configs = [x.split("\\")[6] for x in test_edf_file_list]
list(set(channel_configs))

E:\hy\Datasets\TUSZ\edf\train\01_tcp_ar\000\00000077\s003_2010_01_21\00000077_s003_t000.edf
E:\hy\Datasets\TUSZ\edf\dev\01_tcp_ar\095\00009578\s032_2013_08_25\00009578_s032_t002.edf


['02_tcp_le', '03_tcp_ar_a', '01_tcp_ar']

In [6]:
# 1. Remove test data that exists in train file and update counts
# Read training subjects into set and list
train_subjects = set()
train_subjects_list = []
with open(train_subjects_name_file_path, 'r') as f:
    for line in f:
        line = line.strip()
        if line:
            train_subjects.add(line)
            train_subjects_list.append(line)
original_train_count = len(train_subjects_list)

# Process test data
test_subjects = []
with open(test_subjects_name_file_path, 'r') as f:
    for line in f:
        line = line.strip()
        if line and line not in train_subjects:
            test_subjects.append(line)
test_count = len(test_subjects)

# Write filtered test data back
with open(test_subjects_name_file_path, 'w') as f:
    for subject in test_subjects:
        f.write(f"{subject}\n")

# 2. Create dev set and update train file
import random

# Validate data
if original_train_count == 0:
    raise ValueError("Empty training file")

# Calculate dev size (ensure minimum 1 sample)
dev_size = max(1, round(original_train_count * 0.1))

# Random sampling with reproducibility
random.seed(42)
dev_subjects = random.sample(train_subjects_list, dev_size)
dev_count = len(dev_subjects)

# Create remaining train data (original order preserved)
dev_set = set(dev_subjects)
remaining_train = [s for s in train_subjects_list if s not in dev_set]
updated_train_count = len(remaining_train)

# Update train file with remaining data
with open(train_subjects_name_file_path, 'w') as f:
    for subject in remaining_train:
        f.write(f"{subject}\n")

# Create dev file
dev_file_path = r'E:\hy\Datasets\TUSZ\dev_4_class_corpus_subjects.txt'
with open(dev_file_path, 'w') as f:
    for subject in dev_subjects:
        f.write(f"{subject}\n")

# Print final statistics
print(f"[Data Statistics]")
print(f"Original Train: {original_train_count}")
print(f"Updated Train: {updated_train_count}")
print(f"Test Subjects: {test_count}")
print(f"Dev Subjects: {dev_count}")
print(f"Total (Train+Dev): {updated_train_count + dev_count} (Consistency Check)") 

[Data Statistics]
Original Train: 592
Updated Train: 533
Test Subjects: 31
Dev Subjects: 59
Total (Train+Dev): 592 (Consistency Check)


In [7]:
import os
from glob2 import glob
import mne
import pandas as pd

# Complete the slice of train, test and dev respectively
# subjects_name_file_path = r'E:\hy\Datasets\TUSZ\tarin_4_class_corpus_subjects.txt'
# subjects_name_file_path = r'E:\hy\Datasets\TUSZ\dev_4_class_corpus_subjects.txt'
subjects_name_file_path = r'E:\hy\Datasets\TUSZ\test_4_class_corpus_subjects.txt'
f = open(subjects_name_file_path, 'r')
unique_epilepsy_patient_ids = f.readlines()
unique_epilepsy_patient_ids = [x.strip() for x in unique_epilepsy_patient_ids]

SAMPLING_FREQ = 200.0
WINDOW_LENGTH_SECONDS = 12.0
WINDOW_LENGTH_SAMPLES = int(WINDOW_LENGTH_SECONDS * SAMPLING_FREQ)
raw_label = {"fnsz": 0, "spsz": 1, "cpsz": 2, "gnsz": 3, "absz": 4, "tnsz": 5, "tcsz": 6, "mysz": 7}
label_mapping = {"cfsz": 0, "gnsz": 1, "absz": 2, "ctsz": 3, "no_seiz": 4}
label_count = {label: 0 for label in label_mapping.keys()}

def get_seizure_times(file_name):
    seizure_times = []
    with open(file_name) as f:
        i = 0
        for line in f.readlines():
            for seiz_label in raw_label.keys():
                if seiz_label in line:
                    parts = line.strip().split(" ")
                    # merge label
                    if seiz_label in ["fnsz", "spsz", "cpsz"]:
                        seiz_label = "cfsz"
                    elif seiz_label in ["tnsz", "tcsz", "mysz"]:
                        seiz_label = "ctsz"
                    seizure_times.append([
                        float(parts[0]),
                        float(parts[1]),
                        seiz_label,
                        i
                    ])
                    i+=1
    return seizure_times

dataset_index_rows = []

for idx, patient_id in enumerate(unique_epilepsy_patient_ids):
    print(f"{patient_id} : {idx + 1}/{len(unique_epilepsy_patient_ids)}")

    patient_edf_file_list = glob(f"E:\\hy\\Datasets\\TUSZ\\edf\\*\\*\\*\\*\\*\\{patient_id}_*.edf")
    assert len(patient_edf_file_list) >= 1

    print(len(patient_edf_file_list))

    for raw_file_path in patient_edf_file_list:
        raw_data = mne.io.read_raw_edf(raw_file_path, verbose=False, preload=False)
        file_name = raw_file_path.split('.edf')[0] + ".tse"
        seizure_times = get_seizure_times(file_name)

        for start_sample_index in range(0, int(int(raw_data.times[-1]) * SAMPLING_FREQ), WINDOW_LENGTH_SAMPLES):
            end_sample_index = start_sample_index + WINDOW_LENGTH_SAMPLES

            if end_sample_index > int(int(raw_data.times[-1]) * SAMPLING_FREQ):
                break
            label = "no_seiz"
            for t in seizure_times:
                start_t, end_t, seiz_label, seiz_idx = t
                if not ((end_sample_index <= start_t * SAMPLING_FREQ) or (start_sample_index >= end_t * SAMPLING_FREQ)):
                    label = seiz_label
                    break
                    
            if label_mapping[label] in [0, 1, 2, 3]:
                label_count[label] += 1
                # dataset_index_rows.append(f"{file_name.split('/')[-1]},{label_mapping[label]},{seiz_idx}")

            row = {
                "patient_ID": str(patient_id),
                "raw_file_path": raw_file_path,
                "record_length_seconds": raw_data.times[-1],
                "sampling_freq": SAMPLING_FREQ,
                "channel_config": raw_file_path.split("\\")[6],
                "start_sample_index": start_sample_index,
                "end_sample_index": end_sample_index,
                "label": label,
                "numeric_label": label_mapping[label]
            }
            dataset_index_rows.append(row)


df = pd.DataFrame(dataset_index_rows, columns=["patient_ID", "raw_file_path", "record_length_seconds",
                                               "sampling_freq", "channel_config", "start_sample_index",
                                               "end_sample_index", "label", "numeric_label"])
# Save slice
# df.to_csv("E:\\IGGCN\\data\\tusz_eeg\\4_class\\train_corpus_window_index.csv", index=False, encoding='utf_8_sig')
# df.to_csv("E:\\IGGCN\\data\\tusz_eeg\\4_class\\dev_corpus_window_index.csv", index=False, encoding='utf_8_sig')
df.to_csv("E:\\IGGCN\\data\\tusz_eeg\\4_class\\test_corpus_window_index.csv", index=False, encoding='utf_8_sig')
print("\nsuccess！")
print("seizure：", label_count["cfsz"])
print(label_count["gnsz"])
print(label_count["absz"])
print(label_count["ctsz"])

# Write the results to a file
# with open("all_output_file.txt", "w") as output_file:
#     output_file.write("\n".join(dataset_index_rows))


00001984 : 1/31
1
00010062 : 2/31
25
00004087 : 3/31
7
00002297 : 4/31
8
00005943 : 5/31
10
00010547 : 6/31
1
00009697 : 7/31
37
00005213 : 8/31
1
00004151 : 9/31
4
00004594 : 10/31
2
00005625 : 11/31
2
00003635 : 12/31
1
00000258 : 13/31
2
00002289 : 14/31
1
00009578 : 15/31
48
00009866 : 16/31
23
00010022 : 17/31
46
00001770 : 18/31
4
00000629 : 19/31
2
00000795 : 20/31
2
00001278 : 21/31
1
00003281 : 22/31
1
00001640 : 23/31
1
00010639 : 24/31
10
00005479 : 25/31
27
00010106 : 26/31
17
00009839 : 27/31
68
00010861 : 28/31
1
00000675 : 29/31
2
00003306 : 30/31
1
00005031 : 31/31
1

success！
seizure： 1415
814
79
195


In [8]:
import numpy as np
import pandas as pd
import mne

from eeg_TUSZ_19 import standardize_sensors, downsample, highpass, remove_line_noise, get_brain_waves_power
from tqdm import tqdm  # Import tqdm for progress bars


log_amplitude_feature_matrix = []  # Features used to store logarithmic amplitudes

SAMPLING_FREQ = 200.0  # Sampling frequency
"""
Group recordings by where they are stored. 
You do not want to open recordings repeatedly for different Windows
"""
# index_df = pd.read_csv("E:\\IGGCN\\data\\tusz_eeg\\4_class\\train_corpus_window_index.csv", dtype={"patient_ID": str})
# index_df = pd.read_csv("E:\\IGGCN\\data\\tusz_eeg\\4_class\\dev_corpus_window_index.csv", dtype={"patient_ID": str})
index_df = pd.read_csv("E:\\IGGCN\\data\\tusz_eeg\\4_class\\test_corpus_window_index.csv", dtype={"patient_ID": str})
grouped_df = index_df.groupby("raw_file_path")
adjacency_feature_matrix = np.zeros((index_df.shape[0], 19*19))
# Define a progress bar for the outer loop
outer_loop_bar = tqdm(grouped_df, desc="Processing Files", total=len(grouped_df))


for raw_file_path, group_df in outer_loop_bar:
    print(f"FILE NAME: {raw_file_path}")
    channel_config = str(group_df["channel_config"].unique()[0])

    # Preprocessing = Open file, select channel, apply montage, downsample to 200
    raw_data = mne.io.read_raw_edf(raw_file_path, verbose=True, preload=True)
    raw_data = standardize_sensors(raw_data, channel_config, return_montage=False)
    raw_data, sfreq = downsample(raw_data, SAMPLING_FREQ)

    for window_idx in group_df.index.tolist():
        # Get the raw data of the window
        start_sample = group_df.loc[window_idx]['start_sample_index']
        stop_sample = group_df.loc[window_idx]['end_sample_index']
        window_data = raw_data.get_data(start=start_sample, stop=stop_sample)

        # Fourier transform the window data
        window_data_fft = np.fft.fft(window_data, n=int(SAMPLING_FREQ), axis=-1)

        # only take the positive freq part
        idx_pos = int(np.floor(window_data_fft.shape[1] / 2))
        window_data_fft = window_data_fft[:, :idx_pos]
        amp = np.abs(window_data_fft)
        amp[amp == 0.0] = 1e-8  # avoid log of 0

        # Calculate logarithmic amplitude characteristics
        log_amplitude = np.log(amp)
        log_amplitude_feature_matrix.append(log_amplitude)

        transf_window_data = np.expand_dims(window_data, axis=0)

        # Compute the characteristics of the adjacency matrix (spectral correlation)
        from mne_connectivity import spectral_connectivity_epochs

        for ch_idx in range(19):
            spec_conn = spectral_connectivity_epochs(data=transf_window_data,
                                                     method='coh',
                                                     indices=([ch_idx] * 19, range(19)),
                                                     sfreq=SAMPLING_FREQ,
                                                     fmin=1.0, fmax=40.0,  # 适当设置频率范围
                                                     faverage=True, verbose=False)

            spec_conn_value = spec_conn.get_data()
            assert spec_conn_value.shape[0] == 19
            spec_conn_values = np.squeeze(spec_conn_value)
            start_edge_idx = ch_idx * 19
            end_edge_idx = start_edge_idx + 19
            adjacency_feature_matrix[window_idx, start_edge_idx:end_edge_idx] = spec_conn_values
    # Update the outer loop progress bar
    outer_loop_bar.update(1)
    print(f"\nfile: '{raw_file_path}' [RECORDING] ALL WINDOWS DONE! FILE DONE!...\n")

# Converts a feature matrix to a NumPy array
log_amplitude_feature_matrix = np.array(log_amplitude_feature_matrix)
adjacency_feature_matrix = np.array(adjacency_feature_matrix)

from sklearn.preprocessing import MinMaxScaler
# Normalize the maximum and minimum
min_max_scaler = MinMaxScaler()
log_amplitude_feature_matrix_scaled = min_max_scaler.fit_transform(log_amplitude_feature_matrix.reshape(-1, log_amplitude_feature_matrix.shape[-1])).reshape(log_amplitude_feature_matrix.shape)

adjacency_feature_matrix_scaled = min_max_scaler.fit_transform(adjacency_feature_matrix.reshape(-1, adjacency_feature_matrix.shape[-1])).reshape(adjacency_feature_matrix.shape)

# # Save train
# np.save("E:\\IGGCN\\data\\tusz_eeg\\4_class\\train_feature_matrix.npy", log_amplitude_feature_matrix_scaled)
# np.save("E:\\IGGCN\\data\\tusz_eeg\\4_class\\train_adjacency_feature_matrix.npy", adjacency_feature_matrix_scaled)
# np.save("E:\\IGGCN\\data\\tusz_eeg\\4_class\\train_label_y.npy", index_df["label"].to_numpy())

# # Save dev
# np.save("E:\\IGGCN\\data\\tusz_eeg\\4_class\\dev_feature_matrix.npy", log_amplitude_feature_matrix_scaled)
# np.save("E:\\IGGCN\\data\\tusz_eeg\\4_class\\dev_adjacency_feature_matrix.npy", adjacency_feature_matrix_scaled)
# np.save("E:\\IGGCN\\data\\tusz_eeg\\4_class\\dev_label_y.npy", index_df["label"].to_numpy())

# Save test
np.save("E:\\IGGCN\\data\\tusz_eeg\\4_class\\test_feature_matrix.npy", log_amplitude_feature_matrix_scaled)
np.save("E:\\IGGCN\\data\\tusz_eeg\\4_class\\test_adjacency_feature_matrix.npy", adjacency_feature_matrix_scaled)
np.save("E:\\IGGCN\\data\\tusz_eeg\\4_class\\test_label_y.npy", index_df["label"].to_numpy())

Processing Files:   0%|                                                                        | 0/357 [00:00<?, ?it/s]

FILE NAME: E:\hy\Datasets\TUSZ\edf\dev\01_tcp_ar\095\00009578\s032_2013_08_25\00009578_s032_t002.edf
Extracting EDF parameters from E:\hy\Datasets\TUSZ\edf\dev\01_tcp_ar\095\00009578\s032_2013_08_25\00009578_s032_t002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 76799  =      0.000 ...   299.996 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


Processing Files:   0%|▏                                                             | 1/357 [00:11<1:08:48, 11.60s/it]


file: 'E:\hy\Datasets\TUSZ\edf\dev\01_tcp_ar\095\00009578\s032_2013_08_25\00009578_s032_t002.edf' [RECORDING] ALL WINDOWS DONE! FILE DONE!...

FILE NAME: E:\hy\Datasets\TUSZ\edf\dev\01_tcp_ar\095\00009578\s032_2013_08_25\00009578_s032_t003.edf
Extracting EDF parameters from E:\hy\Datasets\TUSZ\edf\dev\01_tcp_ar\095\00009578\s032_2013_08_25\00009578_s032_t003.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 153855  =      0.000 ...   600.996 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


Processing Files:   1%|▌                                                               | 3/357 [00:29<55:54,  9.48s/it]

...


file: 'E:\hy\Datasets\TUSZ\edf\dev\03_tcp_ar_a\059\00005943\s004_2009_06_29\00005943_s004_t001.edf' [RECORDING] ALL WINDOWS DONE! FILE DONE!...

FILE NAME: E:\hy\Datasets\TUSZ\edf\dev\03_tcp_ar_a\059\00005943\s004_2009_06_29\00005943_s004_t002.edf
Extracting EDF parameters from E:\hy\Datasets\TUSZ\edf\dev\03_tcp_ar_a\059\00005943\s004_2009_06_29\00005943_s004_t002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 169999  =      0.000 ...   424.998 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


Processing Files: 383it [1:45:11,  9.76s/it]


file: 'E:\hy\Datasets\TUSZ\edf\dev\03_tcp_ar_a\059\00005943\s004_2009_06_29\00005943_s004_t002.edf' [RECORDING] ALL WINDOWS DONE! FILE DONE!...

FILE NAME: E:\hy\Datasets\TUSZ\edf\dev\03_tcp_ar_a\059\00005943\s004_2009_06_29\00005943_s004_t003.edf
Extracting EDF parameters from E:\hy\Datasets\TUSZ\edf\dev\03_tcp_ar_a\059\00005943\s004_2009_06_29\00005943_s004_t003.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 169599  =      0.000 ...   423.998 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


Processing Files: 100%|████████████████████████████████████████████████████████████| 357/357 [1:45:22<00:00, 17.71s/it]



file: 'E:\hy\Datasets\TUSZ\edf\dev\03_tcp_ar_a\059\00005943\s004_2009_06_29\00005943_s004_t003.edf' [RECORDING] ALL WINDOWS DONE! FILE DONE!...

